In [1]:
import numpy as np
import tables

In [2]:
parents = [['AD0231-C', 'AD0232-C'], ['AD0254-C', 'AD0255-C'],
           ['AD0305-C', 'AD0306-C'], ['AD0347-C', 'AD0348-C']]

crosses_3L_h5 = '../raw/crosses-3L.h5'

In [3]:
def get_offspring_range(parents, all_parents, samples):
    my_index = samples.index(parents[1])
    start = my_index + 1
    parents_position = all_parents.index(parents)
    if  parents_position == len(all_parents) - 1:
        end = len(samples)
    else:
        end = samples.index(all_parents[parents_position + 1][0])
    return start, end

In [4]:
store_3L = tables.open_file(crosses_3L_h5, 'r')

In [5]:
samples = [x.decode('utf-8') for x in store_3L.get_node('/3L/samples').read()]

In [6]:
def compute_medelian_errors(parents, offspring):
    #autosome only
    mo_alleles = set(parents[0])
    fa_alleles = set(parents[1])
    cnt = cnt_mo = cnt_fa = total = 0
    for ofs in offspring:
        if ofs is None:
            continue
        total += 1
        my_alleles = set(ofs)
        if len(my_alleles & mo_alleles) == 0:
            cnt += 1
            cnt_mo += 1
        if len(my_alleles & fa_alleles) == 0:
            cnt += 1
            cnt_fa += 1
        # Ofspring has more alleles (hz) than parents
        if len(my_alleles) > len(fa_alleles | mo_alleles):
            cnt += 1
    return total, cnt, cnt_mo, cnt_fa

In [7]:
store_3L

File(filename=../raw/crosses-3L.h5, title='', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/3L (Group) ''
/3L/samples (Array(80,)) ''
  atom := StringAtom(itemsize=8, shape=(), dflt=b'')
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := None
/3L/calldata (Group) ''
/3L/calldata/AD (EArray(2501730, 80, 2), shuffle, zlib(1)) ''
  atom := UInt16Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (3276, 10, 2)
/3L/calldata/DP (EArray(2501730, 80), shuffle, zlib(1)) ''
  atom := UInt16Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (6553, 10)
/3L/calldata/GQ (EArray(2501730, 80), shuffle, zlib(1)) ''
  atom := UInt8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := (13107, 10)
/3L/calldata/GT (EArray(2501730, 80), shuffle, zlib(1)